In [ ]:
# jupyter_vim

In [ ]:
# Run Script below
! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic=book --bootstrap-servers=kafka-broker:9092 --schema='{"line": "INTEGER","text": "VARCHAR"}' --source='kafka'

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True

In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT kafka_timestamp, line, text FROM book_view LIMIT 10;")
    values = cur.fetchall()

In [ ]:
pd.DataFrame(values, columns=["timestamp", "line", "text"])

In [ ]:
import duckdb

con = duckdb.connect()
con.sql(
    f"""
    INSTALL postgres;
    LOAD postgres;  
    ATTACH 'host=risingwave port=4566 dbname=dev user=root' AS db (TYPE postgres);  
"""
)
con.sql("SHOW DATABASES;")
con.sql("SHOW ALL TABLES;")